In [1]:
import os
from dotenv import load_dotenv
import weaviate

load_dotenv(override=True)

True

In [2]:
api_key = os.environ["OPENAI_API_KEY"]
weaviate_url = os.environ["WEAVIATE_URL"]

client = weaviate.Client(
    url=weaviate_url,  # Replace with your actual instance (e.g. url="http://localhost:8080/", or "https://your-endpoint.weaviate.network/")
    additional_headers={
        "X-OpenAI-API-Key": api_key
    }
)

/root/TeamMate/.venv/lib/python3.11/site-packages/weaviate/warnings.py:162: DeprecationWarning: Dep016: Python client v3 `weaviate.Client(...)` connections and methods are deprecated. Update
            your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.

            For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
            For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration
            
  warnings.warn(


In [3]:
def search(query):
    print("Received data:", query)
    try:
        print("Querying Weaviate directly...")

        search_results = client.query.get("All_nov_jobs", ["description", "title"]).with_limit(10).with_near_text({"concepts": [query]}).do()
        print("Search results:", search_results)
        
        return search_results

    except Exception as e:
        print(f'Error in /search: {e}')
        return {"error": str(e)}

In [4]:
# Use the search function
query = "Software engineer"
search_results = search(query)

# Print the search results
if 'error' in search_results:
    print("An error occurred:", search_results['error'])
else:
    print("Search results:", search_results)

Received data: Software engineer
Querying Weaviate directly...
Search results: {'data': {'Get': {'All_nov_jobs': [{'description': "A Software Engineer is a highly skilled software developer with expert knowledge in at least one department or system. They work with a high degree of independence and develop information systems to support the business. Engineers are primarily focused on the design or integration of various software systems, databases, and third-party packages.\n\nEssential Job Functions\n\nGenerally, takes leadership role in guiding projects through the complete software development life cycle\nSolid understanding of secure coding techniques and best practices\nResponsible for multiple software applications\nManage tasks and resources while utilizing modern project management principles\nParticipate or lead the development of multiple projects at one time\nConducts code reviews and ensures coding best practices are being followed\nRecommend software improvements and chang

In [7]:
class_name = "all_nov_jobs"
def jprint(json_in):
    import json
    print(json.dumps(json_in, indent=2))

jprint(client.get_meta())

results = client.query.get(
    class_name, ["title", "place", "description"]
).with_near_text(
    {"concepts": ["software engineer"]}
).with_additional(
    ["distance", "id"]
).with_limit(5).do()
jprint(results)

{
  "hostname": "http://[::]:8080",
  "modules": {
    "text2vec-openai": {
      "documentationHref": "https://platform.openai.com/docs/guides/embeddings/what-are-embeddings",
      "name": "OpenAI Module"
    }
  },
  "version": "1.23.0"
}
{
  "data": {
    "Get": {
      "All_nov_jobs": [
        {
          "_additional": {
            "distance": 0.122837186,
            "id": "8bb6d017-cd18-425c-8e6f-26713ce9c9dd"
          },
          "description": "A Software Engineer is a highly skilled software developer with expert knowledge in at least one department or system. They work with a high degree of independence and develop information systems to support the business. Engineers are primarily focused on the design or integration of various software systems, databases, and third-party packages.\n\nEssential Job Functions\n\nGenerally, takes leadership role in guiding projects through the complete software development life cycle\nSolid understanding of secure coding techniques and 

In [8]:
from weaviate.classes.query import MetadataQuery

reviews = client.collections.get("All_nov_jobs")
response = reviews.query.near_text(
    query="a sweet German white wine",
    limit=2,
    target_vector="title_country",  # Specify the target vector for named vector collections
    return_metadata=MetadataQuery(distance=True)
)

for o in response.objects:
    print(o.properties)
    print(o.metadata.distance)

AttributeError: 'Client' object has no attribute 'collections'

# Using langchian

In [ ]:
import weaviate
from langchain_weaviate.vectorstores import WeaviateVectorStore

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.openai import OpenAIEmbeddings

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
weaviate_client =  weaviate.Client(
    url=weaviate_url,  # Replace with your actual instance (e.g. url="http://localhost:8080/", or "https://your-endpoint.weaviate.network/")
    additional_headers={
        "X-OpenAI-API-Key": api_key
    }
)

In [ ]:
import weaviate
from langchain_community.vectorstores import Weaviate

# client = weaviate.Client(url=os.environ["WEAVIATE_URL"])
# weaviate = Weaviate(client, "All_nov_jobs", "title")

In [46]:
from langchain.vectorstores.weaviate import Weaviate
from langchain.llms import OpenAI
from langchain.chains import ChatVectorDBChain
import weaviate

client =  weaviate.Client(
    url=weaviate_url,  # Replace with your actual instance (e.g. url="http://localhost:8080/", or "https://your-endpoint.weaviate.network/")
    additional_headers={
        "X-OpenAI-API-Key": api_key
    })

vectorstore = Weaviate(client, "All_nov_jobs", "title", "description")

MyOpenAI = OpenAI(temperature=0.2,
    openai_api_key=api_key,)

qa = ChatVectorDBChain.from_llm(MyOpenAI, vectorstore)

chat_history = []

while True:
    query = input("")
    result = qa({"question": query, "chat_history": chat_history})
    print(result["answer"])
    chat_history = [(query, result["answer"])]

/root/TeamMate/.venv/lib/python3.11/site-packages/weaviate/warnings.py:162: DeprecationWarning: Dep016: Python client v3 `weaviate.Client(...)` connections and methods are deprecated. Update
            your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.

            For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
            For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration
            
  warnings.warn(
/root/TeamMate/.venv/lib/python3.11/site-packages/langchain/chains/conversational_retrieval/base.py:485: UserWarning: `ChatVectorDBChain` is deprecated - please use `from langchain.chains import ConversationalRetrievalChain`
  warnings.warn(


Welcome to the Weaviate ChatVectorDBChain Demo!
Please enter a question or dialogue to get started!


/root/TeamMate/.venv/lib/python3.11/site-packages/pydantic/main.py:1070: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


 I don't know.


/root/TeamMate/.venv/lib/python3.11/site-packages/pydantic/main.py:1070: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/root/TeamMate/.venv/lib/python3.11/site-packages/pydantic/main.py:1070: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


 Yes, I can find you a software job from the titles. The first two titles are for a software engineer, while the second two titles are for a remote software engineer.


/root/TeamMate/.venv/lib/python3.11/site-packages/pydantic/main.py:1070: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/root/TeamMate/.venv/lib/python3.11/site-packages/pydantic/main.py:1070: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)




I'm sorry, I don't have enough information to provide a specific job description for a software engineering position. However, based on the context provided, it seems like the position may involve working with embedded software and/or backend development using node.js. It may also be a remote position.


/root/TeamMate/.venv/lib/python3.11/site-packages/pydantic/main.py:1070: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/root/TeamMate/.venv/lib/python3.11/site-packages/pydantic/main.py:1070: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


 I don't know.
